<a href="https://colab.research.google.com/github/nizardbsk/pl_pytorch_tutorial/blob/main/14_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
num_epochs = 4
batch_size = 4
lr=0.001

In [4]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./data',train=True,transform=transforms.ToTensor(),download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data',train=False,transform=transforms.ToTensor(),download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')


Files already downloaded and verified
Files already downloaded and verified


In [5]:
def imshow(img):
  img = img/ 2+ 0.5
  npimg = img.numpy()
  plt.imshow(np.transpose(npimg,(1,2,0)))
  plt.show()

In [14]:

# implement the conv net
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet,self).__init__()
    self.conv1 = nn.Conv2d(3,6,5)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(6,16,5)
    self.fc1 = nn.Linear(16*5*5,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)

  def forward(self,x):
    x = self.pool(F.relu(self.conv1(x)))
    x=  self.pool(F.relu(self.conv2(x)))
    x= x.view(-1,16*5*5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [15]:
model = ConvNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=lr)

In [16]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i,(images,labels) in enumerate(train_loader):
    # origin shape : [4,3,32,32] = 4,3,1024
    # input_layer : 3 input channels, 6 output channels, 5 kernel size

    # forward pass
    outputs = model(images)
    loss = criterion(outputs,labels)

    # backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 2000 ==0:
      print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss {loss.item():.4f}')
print('finish training')

epoch 1/4, step 2000/12500, loss 2.2836
epoch 1/4, step 4000/12500, loss 2.2810
epoch 1/4, step 6000/12500, loss 2.3101
epoch 1/4, step 8000/12500, loss 2.3168
epoch 1/4, step 10000/12500, loss 2.2950
epoch 1/4, step 12000/12500, loss 2.2898
epoch 2/4, step 2000/12500, loss 2.3072
epoch 2/4, step 4000/12500, loss 2.2911
epoch 2/4, step 6000/12500, loss 2.3114
epoch 2/4, step 8000/12500, loss 2.2984
epoch 2/4, step 10000/12500, loss 2.2756
epoch 2/4, step 12000/12500, loss 2.2192
epoch 3/4, step 2000/12500, loss 2.3903
epoch 3/4, step 4000/12500, loss 1.7417
epoch 3/4, step 6000/12500, loss 2.3804
epoch 3/4, step 8000/12500, loss 2.1308
epoch 3/4, step 10000/12500, loss 1.5493
epoch 3/4, step 12000/12500, loss 3.6171
epoch 4/4, step 2000/12500, loss 1.7016
epoch 4/4, step 4000/12500, loss 1.9594
epoch 4/4, step 6000/12500, loss 1.7694
epoch 4/4, step 8000/12500, loss 1.1113
epoch 4/4, step 10000/12500, loss 1.7820
epoch 4/4, step 12000/12500, loss 1.8104
finish training


In [20]:
with torch.no_grad() :
  n_correct = 0
  n_samples = 0
  n_class_correct = [ 0 for i in range(10)]
  n_class_samples = [ 0 for i in range(10)]
  for images,labels in test_loader :
    outputs = model(images)
    _,predicted = torch.max(outputs,1)
    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item()

    for i in range(batch_size):
      label = labels[i]
      pred = predicted[i]
      if(label == pred):
        n_class_correct[label] +=1
      n_class_samples[label] +=1
  acc = 100.0 * n_correct /n_samples
  print(f' acc {acc}%')

  for i in range(10):
    acc = 100.0 * n_class_correct[i] / n_class_samples[i]
    print(f'Accuracy of {classes[i]}: {acc} %')

 acc 38.08%
Accuracy of plane: 38.2 %
Accuracy of car: 64.2 %
Accuracy of bird: 3.7 %
Accuracy of cat: 8.0 %
Accuracy of deer: 31.4 %
Accuracy of dog: 39.4 %
Accuracy of frog: 63.7 %
Accuracy of horse: 46.0 %
Accuracy of ship: 46.0 %
Accuracy of truck: 40.2 %
